## Homework: Vector Search (Version 2)

In this homework, we'll experiment with vector search with and without Elasticsearch.

### Q1. Getting the embeddings model

First we will get the embeddings model multi-qa-distilbert-cos-v1 from the [Sentence Transformer library](https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#model-overview)

```
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)
```

Create the embedding for this user question:

```
user_question = "I just discovered the course. Can I still join it?"
```

What is the first value of the resulting vector?

* -0.24
* -0.04
* 0.07
* 0.27

In [1]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("multi-qa-distilbert-cos-v1")

In [2]:
user_question = "I just discovered the course. Can I still join it?"

embeddings = embedding_model.encode(user_question)
embeddings[0]

0.078222655

### A1. 0.07 is closest

For question 3

In [3]:
v = embeddings

### Prepare the documents

Now we will create the embeddings for the documents.

Load the documents with ids that we prepared in the module:



In [4]:
import requests

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [5]:
len(documents)

948

We will use only a subset of the questions - the questions for "machine-learning-zoomcamp".  After filtering, you should have only 375 documents.

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [7]:
filtered_docs = [d for d in documents if d['course'] == 'machine-learning-zoomcamp']
len(filtered_docs)

375

### Q2. Creating the embeddings

Now for each document, we will create an embedding for both the question and answer fields.

We want to put all of them into a single matrix X:

* Create a list of embeddings
* Iterate over each document
* qa_text = f'{question} {text}'
* compute the embedding for qa_text, append it to embeddings
* at the end, let X = np.array(embeddings) (import numpy as np)

What is the shape of X? ( X.shape ). Include the parentheses.

In [8]:
import numpy as np

from sentence_transformers import SentenceTransformer
model = SentenceTransformer("multi-qa-distilbert-cos-v1")

# creating a dense vector using the pre-trained model
embeddings = []
for doc in filtered_docs:
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'
    embeddings.append(model.encode(qa_text).tolist())

In [9]:
X = np.array(embeddings)
X.shape

(375, 768)

### A2. The shape of X is (375,768)

### Q3. Search

We have the embeddings and the query vector.  Now let's compute the cosine similarity between the vector from Q1 (let's call it v) and the matrix from Q2.


The vectors returned from the embedding model are already normalized (you can check it by computing a dot product of a vector with itself - it should return 1.0).  This means that in order to compute the cosine similarity, it's sufficient to multiply the matrix X by the vectort v:

```
scores = X.dot(v)
```

What's the highest score in the results?

* 65.0
* 6.5
* 0.65
* 0.065

In [10]:
scores = X.dot(v)

In [11]:
scores.max()

0.6506573240979582

### A3. The maximum score is 0.65

### Vector search

We can now compute the similarity between a query vector and all the embeddings.

Let's use this to implement our own vector search

In [12]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=filtered_docs, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

If you don't understand how the search function works:

* Ask ChatGPT or any other LLM of your choice to explain the code
* Check our pre-course workshop about implementing a search engine [here](https://github.com/alexeygrigorev/build-your-own-search-engine)

(Note: you can replace argsort with argpartition to make it a lot faster)

### Q4. Hit-rate for our search engine

Let's evaluate the performance of our own search engine. We will use the hit rate metric for evaluation.

Hit Rate (HR) or Recall at k:
```
    Measures the proportion of queries for which at least one relevant document is retrieved in the top k results.
    Formula: HR@k = (Number of queries with at least one relevant document in top k) / |Q|
```

First, load the ground truth dataset:

In [13]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

Now use the code from the module to calculate the hit rate of VectorSearchEngine with num_results=5.

In [14]:
#ground_truth

In [15]:
#if not any(d['question'] == user_question for d in ground_truth):
#    print('Question not found')

In [16]:
#next(iter(ground_truth))

In [17]:
len(ground_truth)

1830

In [18]:
from tqdm.auto import tqdm
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = search_engine.search(v, num_results=5)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/1830 [00:00<?, ?it/s]

In [21]:
len(relevance_total)

1830

In [ ]:
#relevance_total

In [28]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [29]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [30]:
hit_rate(relevance_total)

0.01366120218579235

What did you get?

* 0.93
* 0.73
* 0.53
* 0.33

### A4. The closest answer is 0.33.

### Q5. Indexing with Elasticsearch

Now let's index these documents with elasticsearch

```
    Create the index with the same settings as in the module (but change the dimensions)
    Index the embeddings (note: you've already computed them)
```

In [31]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}


index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [32]:
for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = model.encode(question)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(qt)

  0%|          | 0/948 [00:00<?, ?it/s]

In [33]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

After indexing, let's perform the search of the same query from Q1.
```
user_question = "I just discovered the course. Can I still join it?"
embeddings = embedding_model.encode(user_question)
```

In [34]:
print(user_question)

I just discovered the course. Can I still join it?


In [35]:
vector_search_term = model.encode(user_question)

#### Reference

* https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-simple-query-string-query.html
* https://www.elastic.co/guide/en/elasticsearch/reference/current/query-filter-context.html

In [36]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [37]:
def question_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_vector', vector_search_term, course)

In [38]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [39]:
knn_query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000, 
}

In [40]:
response = es_client.search(
    index=index_name,
    body={
        "query": {
            "bool": {
                "must": [
                    {
                        "script_score": {
                            "query": {"match_all": {}},
                            "script": {
                                "source": """
                                    double cosineSimilarity = cosineSimilarity(params.query_vector, 'question_vector');
                                    if (cosineSimilarity == -1.0) {
                                        return 0.0; // Avoid negative scores, return 0
                                    } else {
                                        return cosineSimilarity + 1.0; // Normalize scores to be positive
                                    }
                                """,
                                "params": {"query_vector": vector_search_term}
                            }
                        }
                    }
                ]
            }
        },
        "size": 5,
        "sort": [
            {"_score": {"order": "desc"}} 
        ],
        "_source": ["id", "question"]
    }
)

In [41]:
response["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': '8eiPw5ABi2qZzrh_BSAa',
  '_score': 1.9345204,
  '_source': {'question': 'The course has already started. Can I still join it?',
   'id': 'ee58a693'}},
 {'_index': 'course-questions',
  '_id': 'MuiOw5ABi2qZzrh_3h86',
  '_score': 1.7614222,
  '_source': {'question': 'Course - Can I still join the course after the start date?',
   'id': '7842b56a'}},
 {'_index': 'course-questions',
  '_id': 'N-iOw5ABi2qZzrh_3h-r',
  '_score': 1.7263663,
  '_source': {'question': 'Course - Can I follow the course after it finishes?',
   'id': 'a482086d'}},
 {'_index': 'course-questions',
  '_id': '9OiPw5ABi2qZzrh_BSBf',
  '_score': 1.6746192,
  '_source': {'question': 'I just joined. What should I do next? How can I access course materials?',
   'id': '0a278fb2'}},
 {'_index': 'course-questions',
  '_id': 'MOiOw5ABi2qZzrh_3h8F',
  '_score': 1.6439786,
  '_source': {'question': 'Course - When will the course start?',
   'id': 'c02e79ef'}}]

What's the ID of the document with the highest score?

### A5. The ID of the document with the highest score that we assigned is ee58a693.

### Q6. Hit-rate for Elasticsearch

The search engine we used in Q4 computed the similarity between the query and ALL the vectors in our database. Usually this is not practical, as we may have a lot of data.

Elasticsearch uses approximate techniques to make it faster.

Let's evaluate how worse the results are when we switch from exact search (as in Q4) to approximate search with Elastic.

In [42]:
evaluate(ground_truth, question_vector_knn)

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.01366120218579235, 'mrr': 0.006238615664845174}

What's hit rate for our dataset for Elastic?

* 0.93
* 0.73
* 0.53
* 0.33

### A6.  The closest answer is 0.33.